In [2]:
import os
import math
import wandb
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt


from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras.metrics import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *

from tensorflow.keras.regularizers import *

from tensorflow.keras.callbacks import *
from keras.preprocessing.image import *
from tensorflow.keras.preprocessing import *

2023-05-21 11:57:49.095766: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-21 11:57:49.216498: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

# Normalize pixel values between 0 and 1
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert labels to one-hot encoded vectors
y_train = tf.keras.utils.to_categorical(y_train, num_classes=100)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=100)

In [72]:
def building_block(X, filter_size, filters, stride=1):

    # Save the input value for shortcut
    X_shortcut = X

    # Reshape shortcut for later adding if dimensions change
    if stride > 1:

        X_shortcut = Conv2D(filters, (1, 1), strides=stride, padding='same')(X_shortcut)
        X_shortcut = BatchNormalization(axis=3)(X_shortcut)

    # First layer of the block
    X = Conv2D(filters, kernel_size = filter_size, strides=stride, padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Second layer of the block
    X = Conv2D(filters, kernel_size = filter_size, strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = add([X, X_shortcut])  # Add shortcut value to main path
    X = Activation('relu')(X)

    return X

def ResNet32(input_shape, classes, name):

    # Define the input
    X_input = Input(input_shape)

    # Stage 1
    X = Conv2D(filters=16, kernel_size=3, strides=(1, 1), padding='same')(X_input)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Stage 2
    X = building_block(X, filter_size=3, filters=16, stride=1)
    X = building_block(X, filter_size=3, filters=16, stride=1)
    X = building_block(X, filter_size=3, filters=16, stride=1)
    X = building_block(X, filter_size=3, filters=16, stride=1)
    X = building_block(X, filter_size=3, filters=16, stride=1)

    # Stage 3
    X = building_block(X, filter_size=3, filters=32, stride=2)  # dimensions change (stride=2)
    X = building_block(X, filter_size=3, filters=32, stride=1)
    X = building_block(X, filter_size=3, filters=32, stride=1)
    X = building_block(X, filter_size=3, filters=32, stride=1)
    X = building_block(X, filter_size=3, filters=32, stride=1)

    # Stage 4
    X = building_block(X, filter_size=3, filters=64, stride=2)  # dimensions change (stride=2)
    X = building_block(X, filter_size=3, filters=64, stride=1)
    X = building_block(X, filter_size=3, filters=64, stride=1)
    X = building_block(X, filter_size=3, filters=64, stride=1)
    X = building_block(X, filter_size=3, filters=64, stride=1)

    # Average pooling and output layer
    X = GlobalAveragePooling2D()(X)
    X = Dense(classes, activation='softmax')(X)

    # Create model
    model = Model(inputs=X_input, outputs=X, name=name)

    return model

In [73]:
input_shape = list(x_train.shape[1:])
classes = y_train.shape[1]
model = ResNet32(input_shape=input_shape, classes=classes, name='ResNet32')

In [74]:
# 옵티마이저 및 학습률 스케줄러 정의
# 옵티마이저 정의
learning_rate = 1e-3

def learning_rate_schedule(epoch):
    new_learning_rate = learning_rate

    if epoch <= 50:
        pass
    elif epoch > 50 and epoch <= 100:
        new_learning_rate = learning_rate * 0.1
    else:
        new_learning_rate = learning_rate * 0.01
        
    print('Learning rate:', new_learning_rate)
    
    return new_learning_rate


opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)


model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [75]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(horizontal_flip=True,
                             vertical_flip= True
                             )

# 이미지 데이터의 분포를 조정
datagen.fit(x_train)

In [76]:
from tensorflow.keras import callbacks

wandb.init(project="CIFAR100", entity="hcim", name='ResNet-relu')

wandbCallback=wandb.keras.WandbCallback(monitor="val_accuracy", mode="max", 
                                        log_weights=True, log_gradients=True, 
                                        training_data=datagen.flow(x_train, y_train, batch_size=128))


callbacks = [wandbCallback, LearningRateScheduler(learning_rate_schedule)]

In [77]:
model.fit(datagen.flow(x_train, y_train, batch_size=128),
          validation_data=(x_test, y_test),
          epochs=200, workers=4,
          callbacks=callbacks)

Learning rate: 0.001
Epoch 1/200
391/391 [==============================] - ETA: 0s - loss: 4.0045 - accuracy: 0.0846

wandb: Adding directory to artifact (/root/jupyter/IHC/gain_proj/ResNet50_CIFAR/CIFAR_100/wandb/run-20230522_021825-ptsv149v/files/model-best)... Done. 0.1s


391/391 [==============================] - 66s 161ms/step - loss: 4.0045 - accuracy: 0.0846 - val_loss: 4.2711 - val_accuracy: 0.0648 - lr: 0.0010
Learning rate: 0.001
Epoch 2/200
390/391 [============================>.] - ETA: 0s - loss: 3.3884 - accuracy: 0.1748

wandb: Adding directory to artifact (/root/jupyter/IHC/gain_proj/ResNet50_CIFAR/CIFAR_100/wandb/run-20230522_021825-ptsv149v/files/model-best)... Done. 0.1s


391/391 [==============================] - 59s 152ms/step - loss: 3.3874 - accuracy: 0.1751 - val_loss: 3.5464 - val_accuracy: 0.1572 - lr: 0.0010
Learning rate: 0.001
Epoch 3/200
390/391 [============================>.] - ETA: 0s - loss: 2.9810 - accuracy: 0.2485

...

391/391 [==============================] - 60s 153ms/step - loss: 0.7204 - accuracy: 0.7857 - val_loss: 1.5189 - val_accuracy: 0.5923 - lr: 1.0000e-04
Learning rate: 0.0001
Epoch 55/200
390/391 [============================>.] - ETA: 0s - loss: 0.7114 - accuracy: 0.7881

wandb: Adding directory to artifact (/root/jupyter/IHC/gain_proj/ResNet50_CIFAR/CIFAR_100/wandb/run-20230522_021825-ptsv149v/files/model-best)... Done. 0.1s


391/391 [==============================] - 59s 151ms/step - loss: 0.7114 - accuracy: 0.7881 - val_loss: 1.5235 - val_accuracy: 0.5933 - lr: 1.0000e-04
Learning rate: 0.0001
Epoch 56/200
391/391 [==============================] - 49s 127ms/step - loss: 0.7004 - accuracy: 0.7910 - val_loss: 1.5302 - val_accuracy: 0.5899 - lr: 1.0000e-04
Learning rate: 0.0001
Epoch 57/200
390/391 [============================>.] - ETA: 0s - loss: 0.6909 - accuracy: 0.7956

wandb: Adding directory to artifact (/root/jupyter/IHC/gain_proj/ResNet50_CIFAR/CIFAR_100/wandb/run-20230522_021825-ptsv149v/files/model-best)... Done. 0.1s


391/391 [==============================] - 60s 154ms/step - loss: 0.6909 - accuracy: 0.7956 - val_loss: 1.5237 - val_accuracy: 0.5947 - lr: 1.0000e-04
Learning rate: 0.0001
Epoch 58/200
391/391 [==============================] - 50s 127ms/step - loss: 0.6854 - accuracy: 0.7960 - val_loss: 1.5334 - val_accuracy: 0.5937 - lr: 1.0000e-04
Learning rate: 0.0001
Epoch 59/200
391/391 [==============================] - 50s 128ms/step - loss: 0.6836 - accuracy: 0.7953 - val_loss: 1.5463 - val_accuracy: 0.5911 - lr: 1.0000e-04
Learning rate: 0.0001
Epoch 60/200
391/391 [==============================] - 50s 127ms/step - loss: 0.6734 - accuracy: 0.8005 - val_loss: 1.5556 - val_accuracy: 0.5916 - lr: 1.0000e-04
Learning rate: 0.0001
Epoch 61/200
391/391 [==============================] - 50s 127ms/step - loss: 0.6692 - accuracy: 0.8002 - val_loss: 1.5414 - val_accuracy: 0.5897 - lr: 1.0000e-04
Learning rate: 0.0001
Epoch 62/200
391/391 [==============================] - 50s 127ms/step - loss: 0.66